In [1]:
from z3 import *

# FUNZIONI UTILITY

def check_and_print(solver):
    """Controlla se il solver ha una soluzione e la stampa."""
    has_solution = solver.check()
    if has_solution == sat:
        print(f"\nThe constraints are satisfiable.", end=' ')
        print(f"Solution: \n\t{solver.model()}")
        return sat
    else:
        print(f"\nThe constraints are unsatisfiable")
        return unsat


def op_to_symbol(op):
    """Converte l'indice dell'operazione nel simbolo corrispondente."""
    return ['+', '-', '*', '/'][op]


def apply_op(a, b, op):
    """
    Applica l'operazione specificata a due numeri.
    op: 0=addizione, 1=sottrazione, 2=moltiplicazione, 3=divisione
    Ritorna None se l'operazione non è valida (es. divisione con resto).
    """
    if op == 0:
        return a + b
    elif op == 1:
        return a - b
    elif op == 2:
        return a * b
    elif op == 3:
        # Divisione valida solo se b != 0 e divisione esatta
        if b != 0 and a % b == 0:
            return a // b
    return None

# PARTE BASE: STRATEGIA STANDARD

def CountingStrategy(numbers, goal):
    """
    Trova la migliore strategia per raggiungere il goal usando i numeri disponibili.
    Prova con un numero crescente di passi (da 2 a len(numbers)).
    Si ferma quando trova una soluzione esatta (distanza = 0).
    
    Args:
        numbers: lista di numeri disponibili
        goal: numero obiettivo da raggiungere
    
    Returns:
        La migliore soluzione trovata
    """
    best_solution = None
    best_distance = float('inf')
    
    # Prova con k numeri, dove k va da 2 a len(numbers)
    for k in range(2, len(numbers) + 1):
        print(f"\nTrying with {k} numbers...")
        
        # Trova tutte le soluzioni con k numeri
        solutions = solve_k_steps(numbers, goal, k)
        
        # Aggiorna la migliore soluzione trovata
        for sol in solutions:
            distance = abs(sol['final'] - goal)
            if distance < best_distance:
                best_distance = distance
                best_solution = sol
        
        # Se abbiamo trovato una soluzione esatta, fermiamoci
        if best_distance == 0:
            break
    
    # Stampa la soluzione trovata
    if best_solution:
        print_solution(best_solution, goal)
    
    return best_solution


def solve_k_steps(numbers, goal, k):
    """
    Trova tutte le soluzioni che usano esattamente k numeri.
    Minimizza la distanza dal goal.
    
    Args:
        numbers: lista di numeri disponibili
        goal: numero obiettivo
        k: numero di passi da usare
    
    Returns:
        Lista di soluzioni ordinate per distanza crescente
    """
    n = len(numbers)
    
    # Variabili
    idx = [Int(f'idx_{i}') for i in range(k)]      # Indici dei numeri scelti
    op = [Int(f'op_{i}') for i in range(k - 1)]    # Operazioni tra i numeri
    res = [Int(f'res_{i}') for i in range(k)]      # Risultati intermedi
    
    s = Solver()
    
    # VINCOLI: Indici validi e tutti diversi
    for i in range(k):
        s.add(idx[i] >= 0, idx[i] < n)
    s.add(Distinct(idx))  # Ogni numero può essere usato una sola volta
    
    # VINCOLI: Operazioni valide (0=+, 1=-, 2=*, 3=/)
    for i in range(k - 1):
        s.add(op[i] >= 0, op[i] <= 3)
    
    # VINCOLO: Primo risultato = primo numero scelto
    for i in range(n):
        s.add(Implies(idx[0] == i, res[0] == numbers[i]))
    
    # VINCOLI: Transizioni tra uno step e il successivo
    for step in range(k - 1):
        for i in range(n):
            num = numbers[i]
            # Addizione
            s.add(Implies(And(idx[step + 1] == i, op[step] == 0), 
                         res[step + 1] == res[step] + num))
            # Sottrazione
            s.add(Implies(And(idx[step + 1] == i, op[step] == 1), 
                         res[step + 1] == res[step] - num))
            # Moltiplicazione
            s.add(Implies(And(idx[step + 1] == i, op[step] == 2), 
                         res[step + 1] == res[step] * num))
            # Divisione (solo se esatta)
            s.add(Implies(And(idx[step + 1] == i, op[step] == 3), 
                         And(num != 0, res[step] == res[step + 1] * num)))
    
    # Calcolo della distanza dal goal
    distance = Int('distance')
    final = res[k - 1]
    s.add(Or(
        And(final >= goal, distance == final - goal),
        And(final < goal, distance == goal - final)
    ))
    
    solutions = []
    
    # Trova tutte le soluzioni, minimizzando la distanza
    while s.check() == sat:
        m = s.model()
        
        # Estrai i valori dalla soluzione
        sol_idx = [m[idx[i]].as_long() for i in range(k)]
        sol_op = [m[op[i]].as_long() for i in range(k - 1)]
        sol_res = [m[res[i]].as_long() for i in range(k)]
        sol_distance = m[distance].as_long()
        
        # Crea il dizionario soluzione
        sol = {
            'indices': sol_idx,
            'numbers': [numbers[i] for i in sol_idx],
            'ops': sol_op,
            'results': sol_res,
            'initial': numbers[sol_idx[0]],
            'final': sol_res[-1],
            'distance': sol_distance
        }
        solutions.append(sol)
        
        # Cerca soluzioni con distanza minore
        s.add(distance < sol_distance)
    
    return solutions


def print_solution(solution, goal):
    """Stampa in modo leggibile una soluzione."""
    print(f"\nInitial number: {solution['initial']}")
    
    for i, op in enumerate(solution['ops']):
        num = solution['numbers'][i + 1]
        result = solution['results'][i + 1]
        print(f"Step {i+1}: {op_to_symbol(op)} {num} → {result}")
    
    print(f"Final number: {solution['final']}")
    print(f"Distance from goal: {solution['distance']}")


# PARTE BONUS: STRATEGIA RESILIENTE AGLI ATTACCHI

def CountingStrategyResilient(numbers, goal):
    """
    Trova la strategia più resiliente agli attacchi.
    L'ultimo numero può essere sostituito da un attaccante con un numero da 0 a 10.
    Cerca di minimizzare la distanza peggiore possibile dopo l'attacco.
    
    Args:
        numbers: lista di numeri disponibili
        goal: numero obiettivo
    
    Returns:
        La migliore soluzione resiliente trovata
    """
    best_solution = None
    best_worst_distance = float('inf')
    
    # Prova con k numeri, dove k va da 2 a len(numbers)
    for k in range(2, len(numbers) + 1):
        print(f"\nTrying with {k} numbers...")
        
        solutions = solve_k_steps_resilient(numbers, goal, k)
        
        # Trova la soluzione con la minore distanza nel caso peggiore
        for sol in solutions:
            if sol['worst_distance'] < best_worst_distance:
                best_worst_distance = sol['worst_distance']
                best_solution = sol
    
    if best_solution:
        print_solution_resilient(best_solution, goal)
    
    return best_solution


def solve_k_steps_resilient(numbers, goal, k):
    """
    Trova soluzioni alternativa.
    Considera che l'ultimo numero possa essere sostituito da 0 a 10.
    Minimizza la distanza nel caso peggiore.
    
    Args:
        numbers: lista di numeri disponibili
        goal: numero obiettivo
        k: numero di passi da usare
    
    Returns:
        Lista di soluzioni ordinate per worst_distance crescente
    """
    n = len(numbers)
    
    # Variabili 
    idx = [Int(f'idx_{i}') for i in range(k)]
    op = [Int(f'op_{i}') for i in range(k - 1)]
    res = [Int(f'res_{i}') for i in range(k - 1)]  # Solo fino al penultimo step
    
    # Variabili per ogni possibile attaccante (0-10)
    final_att = [Int(f'final_att_{a}') for a in range(11)]
    dist_att = [Int(f'dist_att_{a}') for a in range(11)]
    worst_distance = Int('worst_distance')
    
    s = Solver()
    
    # VINCOLI: Indici validi e tutti diversi
    for i in range(k):
        s.add(idx[i] >= 0, idx[i] < n)
    s.add(Distinct(idx))
    
    # VINCOLI: Operazioni valide
    for i in range(k - 1):
        s.add(op[i] >= 0, op[i] <= 3)
    
    # VINCOLO: Primo risultato
    for i in range(n):
        s.add(Implies(idx[0] == i, res[0] == numbers[i]))
    
    # VINCOLI: Transizioni fino al penultimo step
    for step in range(k - 2):
        for i in range(n):
            num = numbers[i]
            s.add(Implies(And(idx[step + 1] == i, op[step] == 0), 
                         res[step + 1] == res[step] + num))
            s.add(Implies(And(idx[step + 1] == i, op[step] == 1), 
                         res[step + 1] == res[step] - num))
            s.add(Implies(And(idx[step + 1] == i, op[step] == 2), 
                         res[step + 1] == res[step] * num))
            s.add(Implies(And(idx[step + 1] == i, op[step] == 3), 
                         And(num != 0, res[step] == res[step + 1] * num)))
    
    # VINCOLI: Ultimo step con ogni possibile attaccante
    prev_res = res[k - 2]
    last_op = op[k - 2]
    
    for a in range(11):
        # Risultato finale con attaccante = a
        s.add(Implies(last_op == 0, final_att[a] == prev_res + a))
        s.add(Implies(last_op == 1, final_att[a] == prev_res - a))
        s.add(Implies(last_op == 2, final_att[a] == prev_res * a))
        if a != 0:
            s.add(Implies(last_op == 3, prev_res == final_att[a] * a))
        else:
            s.add(Implies(last_op == 3, final_att[a] == 0))
        
        # Distanza per ogni attaccante
        s.add(Or(
            And(final_att[a] >= goal, dist_att[a] == final_att[a] - goal),
            And(final_att[a] < goal, dist_att[a] == goal - final_att[a])
        ))
        s.add(dist_att[a] >= 0)
        
        # La worst_distance deve essere >= di tutte le distanze
        s.add(worst_distance >= dist_att[a])
    
    s.add(worst_distance >= 0)
    
    solutions = []
    
    # Trova tutte le soluzioni, minimizzando worst_distance
    while s.check() == sat:
        m = s.model()
        
        # Estrai i valori
        sol_idx = [m[idx[i]].as_long() for i in range(k)]
        sol_op = [m[op[i]].as_long() for i in range(k - 1)]
        sol_res = [m[res[i]].as_long() for i in range(k - 1)]
        sol_worst = m[worst_distance].as_long()
        
        # Calcola quale attaccante causa la distanza peggiore
        prev_result = sol_res[-1]
        last_operation = sol_op[-1]
        
        worst_att = 0
        worst_final = prev_result
        max_dist = 0
        
        for a in range(11):
            final = apply_op(prev_result, a, last_operation)
            if final is not None:
                dist = abs(final - goal)
                if dist > max_dist:
                    max_dist = dist
                    worst_att = a
                    worst_final = final
        
        # Risultato normale (senza attacco)
        player_final = apply_op(prev_result, numbers[sol_idx[-1]], last_operation)
        full_results = list(sol_res) + [player_final if player_final else 0]
        
        # Crea il dizionario soluzione
        sol = {
            'indices': sol_idx,
            'numbers': [numbers[i] for i in sol_idx],
            'ops': sol_op,
            'results': full_results,
            'initial': numbers[sol_idx[0]],
            'final': full_results[-1],
            'prev_result': prev_result,
            'last_op': last_operation,
            'worst_attacker': worst_att,
            'worst_final': worst_final,
            'worst_distance': max_dist
        }
        solutions.append(sol)
        
        # Cerca soluzioni con worst_distance minore
        s.add(worst_distance < sol_worst)
    
    return solutions


def print_solution_resilient(solution, goal):
    """Stampa in modo leggibile una soluzione resiliente."""
    print(f"\nInitial number: {solution['initial']}")
    
    # Stampa tutti gli step tranne l'ultimo
    for i in range(len(solution['ops']) - 1):
        num = solution['numbers'][i + 1]
        result = solution['results'][i + 1]
        print(f"Step {i+1}: {op_to_symbol(solution['ops'][i])} {num} → {result}")
    
    # Stampa l'ultimo step
    last_step = len(solution['ops'])
    last_num = solution['numbers'][-1]
    last_result = solution['results'][-1]
    print(f"Step {last_step}: {op_to_symbol(solution['last_op'])} {last_num} → {last_result}")
    
    print(f"Final number: {solution['final']}")
    print(f"Distance from goal: {abs(solution['final'] - goal)}")
    print(f"Worst attacker: {solution['worst_attacker']}")
    print(f"Distance after worst attack: {solution['worst_distance']}")


if __name__ == "__main__":
    
    print("=" * 60)
    print("Test 1: CountingStrategy([1, 3, 5, 8, 10, 50], 462)")
    print("=" * 60)
    CountingStrategy([1, 3, 5, 8, 10, 50], 462)
    
    print("\n\n" + "=" * 60)
    print("Test 6: CountingStrategyResilient([1, 3, 5, 8, 10, 50], 462)")
    print("=" * 60)
    CountingStrategyResilient([1, 3, 5, 8, 10, 50], 462)

Test 1: CountingStrategy([1, 3, 5, 8, 10, 50], 462)

Trying with 2 numbers...

Trying with 3 numbers...

Trying with 4 numbers...

Initial number: 50
Step 1: - 3 → 47
Step 2: * 10 → 470
Step 3: - 8 → 462
Final number: 462
Distance from goal: 0


Test 6: CountingStrategyResilient([1, 3, 5, 8, 10, 50], 462)

Trying with 2 numbers...

Trying with 3 numbers...

Trying with 4 numbers...

Trying with 5 numbers...

Trying with 6 numbers...

Initial number: 50
Step 1: - 5 → 45
Step 2: * 10 → 450
Step 3: - 1 → 449
Step 4: + 8 → 457
Step 5: + 3 → 460
Final number: 460
Distance from goal: 2
Worst attacker: 0
Distance after worst attack: 5
